In [27]:
import requests
import time
import json
import locale

from datetime import date, datetime, timedelta

import pandas as pd
from bs4 import BeautifulSoup

# –î–æ–º–∞—à–Ω–µ–µ –∑–∞–¥–∞–Ω–∏–µ –∫ –ª–µ–∫—Ü–∏–∏ "–û—Å–Ω–æ–≤—ã –≤–µ–±-—Å–∫—Ä–∞–ø–∏–Ω–≥–∞ –∏ —Ä–∞–±–æ—Ç—ã —Å API"

## –ó–∞–¥–∞–Ω–∏–µ 1. 

### –û–±—è–∑–∞—Ç–µ–ª—å–Ω–∞—è —á–∞—Å—Ç—å

–ë—É–¥–µ–º –ø–∞—Ä—Å–∏—Ç—å —Å—Ç—Ä–∞–Ω–∏—Ü—É —Å–æ —Å–≤–µ–∂–µ–º–∏ –Ω–æ–≤–æ—Å—Ç—è–º –Ω–∞ [habr.com/ru/all/](https://habr.com/ru/all/).

–í–∞–º –Ω–µ–æ–±—Ö–æ–¥–∏–º–æ —Å–æ–±–∏—Ä–∞—Ç—å —Ç–æ–ª—å–∫–æ —Ç–µ —Å—Ç–∞—Ç—å–∏, –≤ –∫–æ—Ç–æ—Ä—ã—Ö –≤—Å—Ç—Ä–µ—á–∞–µ—Ç—Å—è —Ö–æ—Ç—è –±—ã –æ–¥–Ω–æ —Ç—Ä–µ–±—É–µ–º–æ–µ –∫–ª—é—á–µ–≤–æ–µ —Å–ª–æ–≤–æ. –≠—Ç–∏ —Å–ª–æ–≤–∞ –æ–ø—Ä–µ–¥–µ–ª—è–µ–º –≤ –Ω–∞—á–∞–ª–µ –∫–æ–¥–∞ –≤ –ø–µ—Ä–µ–º–µ–Ω–Ω–æ–π, –Ω–∞–ø—Ä–∏–º–µ—Ä:

`KEYWORDS = ['python', '–ø–∞—Ä—Å–∏–Ω–≥']`

 –ü–æ–∏—Å–∫ –≤–µ—Å—Ç–∏ –ø–æ –≤—Å–µ–π –¥–æ—Å—Ç—É–ø–Ω–æ–π preview-–∏–Ω—Ñ–æ—Ä–º–∞—Ü–∏–∏ (—ç—Ç–æ –∏–Ω—Ñ–æ—Ä–º–∞—Ü–∏—è, –¥–æ—Å—Ç—É–ø–Ω–∞—è –Ω–µ–ø–æ—Å—Ä–µ–¥—Å—Ç–≤–µ–Ω–Ω–æ —Å —Ç–µ–∫—É—â–µ–π —Å—Ç—Ä–∞–Ω–∏—Ü—ã). 
 
–í –∏—Ç–æ–≥–µ –¥–æ–ª–∂–µ–Ω —Ñ–æ—Ä–º–∏—Ä–æ–≤–∞—Ç—å—Å—è –¥–∞—Ç–∞—Ñ—Ä–µ–π–º –≤–∏–¥–∞: `<–¥–∞—Ç–∞> - <–∑–∞–≥–æ–ª–æ–≤–æ–∫> - <—Å—Å—ã–ª–∫–∞>`

In [85]:
PAGES = 10
KEYWORDS = set(['python', 'c++', '–ø–∞—Ä—Å–∏–Ω–≥', 'apple', 'linux', 'windows'])

In [86]:
posts = []
for i in range(0, PAGES):
    req = requests.get('https://habr.com/ru/all/' + ('page%d/' % i if i > 0 else ''))
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = posts + [post for post in soup.find_all('article', class_='post')]

In [29]:
locale.setlocale(locale.LC_TIME, ('ru', 'UTF-8'))

'ru_RU.UTF-8'

In [87]:
def parse_date(date_str):
    result = None
    date_parts = date_str.split(' –≤ ')
    if date_parts[0] == '—Å–µ–≥–æ–¥–Ω—è':
        result = datetime.now()
    elif date_parts[0] == '–≤—á–µ—Ä–∞':
        result = datetime.now() - timedelta(days=1)
    else:
        result = datetime.strptime(date_parts[0], '%d %B %Y')
    hh, mm = date_parts[1].split(':')
    result = result.replace(hour=int(hh), minute=int(mm), second=0, microsecond=0)
    return result

In [88]:
def parse_post(post):
    post_date = parse_date(post.find('span', class_='post__time').text)
    post_link = post.find('a', class_='post__title_link').get('href')
    post_title = post.find('h2', class_='post__title').text
    post_preview = post.find('div', class_='post__text').text
    return {'date': post_date, 'title': post_title, 'link': post_link, 'text': post_preview}

In [89]:
def filter_posts(posts):
    result = pd.DataFrame()
    for post in posts:
        post = parse_post(post)
        if set(post['text'].lower().split(' ')).intersection(KEYWORDS):
            result = pd.concat([result, pd.DataFrame([post])])
    return result

In [90]:
result = filter_posts(posts)
result

date                                              title  \
0 2021-02-10 12:36:00  \n–ü—Ä–µ–≤—Ä–∞—â–∞–µ–º —Å—Ç–∞—Ä—ã–π Amazon Kindle –≤ –ø–ª–∞—Ç—Ñ–æ—Ä–º—É ...   
0 2021-02-10 11:30:00              \n–°—Ä–∞–∂–µ–Ω–∏–µ Linux –Ω–∞ –ø–æ–ø—Ä–∏—â–µ Windows\n   
0 2021-02-10 11:00:00  \nApple M1 —Ö–æ—Ä–æ—à, –Ω–æ –Ω–∞—Å–∫–æ–ª—å–∫–æ –æ–Ω —Ö–æ—Ä–æ—à –≤ —Ç—Ä–∞—Å...   
0 2021-02-10 12:36:00  \n–ü—Ä–µ–≤—Ä–∞—â–∞–µ–º —Å—Ç–∞—Ä—ã–π Amazon Kindle –≤ –ø–ª–∞—Ç—Ñ–æ—Ä–º—É ...   
0 2021-02-10 11:30:00              \n–°—Ä–∞–∂–µ–Ω–∏–µ Linux –Ω–∞ –ø–æ–ø—Ä–∏—â–µ Windows\n   
0 2021-02-10 11:00:00  \nApple M1 —Ö–æ—Ä–æ—à, –Ω–æ –Ω–∞—Å–∫–æ–ª—å–∫–æ –æ–Ω —Ö–æ—Ä–æ—à –≤ —Ç—Ä–∞—Å...   
0 2021-02-09 19:21:00              \n–°–±—Ä–æ—Å –ø—Ä–æ–±–µ–≥–∞ –¥–ª—è RICOH SP 150SUw\n   
0 2021-02-09 14:53:00  \nRoot cause –∞–Ω–∞–ª–∏–∑ –∏–Ω—Ü–∏–¥–µ–Ω—Ç–æ–≤ –Ω–∞ –∫–æ—Ä—Ä–µ–ª—è—Ü–∏—è—Ö ...   
0 2021-02-09 12:12:00  \n20 –ª–µ—Ç Vide–æLAN Client Player: 8 –Ω–µ–æ—á–µ–≤–∏–¥–Ω—ã—Ö...   
0 2021-02-09 10:59:00  \n11 –¥—Ä—É–∑–µ–π Sanic‚Äô–∞ ‚Äì —Å–æ–±–∏—Ä–∞–µ–º –∞—Å–∏–Ω—Ö—Ä–æ–Ω–Ω–æ–µ –≤–µ–±...   
0 2021-02-09 00:53:00                        \n–ß—Ç–æ —Ç–∞–∫–æ–µ —Ñ–∏–ª—å—Ç—Ä –ë–ª—É–º–∞?\n   
0 2021-02-08 12:45:00  \n–ü–æ–ª–æ–∂–µ–Ω–∏–µ –¥–µ–ª —É Windows: —Å–∫–æ–ª—å–∫–æ —Ä–∞–∑–Ω–æ—à—ë—Ä—Å—Ç–Ω...   
0 2021-02-08 12:01:00  \n–ñ—É–ª–∏–∫–∏ –ø—Ä–æ—Ç–∏–≤ —Ä–∞–∑—Ä–∞–±–æ—Ç—á–∏–∫–æ–≤ –ø—Ä–∏–ª–æ–∂–µ–Ω–∏–π Apple...   
0 2021-02-08 11:47:00                             \nMac –≤ –¥–∞—Ç–∞-—Ü–µ–Ω—Ç—Ä–∞—Ö\n   
0 2021-02-08 09:58:00        \n–¢—Ä–µ–Ω–¥—ã 2021 –≤ –∏–Ω–¥—É—Å—Ç—Ä–∏–∏ –ø–æ–ª—É–ø—Ä–æ–≤–æ–¥–Ω–∏–∫–æ–≤\n   
0 2021-02-08 03:45:00  \n–ù–∞ GitHub –ø—Ä–µ–¥–ª–∞–≥–∞—é—Ç –∑–∞–ø—É—Å—Ç–∏—Ç—å –∫–∞—Ç–∞–ª–æ–≥ –º–æ–±–∏–ª...   
0 2021-02-07 21:06:00  \n–°–µ–∫—Ä–µ—Ç–Ω—ã–π —Å–æ–ø—Ä–æ—Ü–µ—Å—Å–æ—Ä Apple M1: —á—Ç–æ —ç—Ç–æ, –∑–∞—á...   
0 2021-02-07 20:00:00  \n–ê–Ω–æ–Ω—Å. –ö–∞–∫ –Ω–∞—É—á–∏—Ç—å—Å—è –ø—Ä–æ—Ö–æ–¥–∏—Ç—å —Å–æ–±–µ—Å–µ–¥–æ–≤–∞–Ω–∏–µ...   

                                                link  \
0    https://habr.com/ru/company/vdsina/blog/541616/   
0                   https://habr.com/ru/post/541730/   
0   https://habr.com/ru/company/pixonic/blog/541622/   
0    https://habr.com/ru/company/vdsina/blog/541616/   
0                   https://habr.com/ru/post/541730/   
0   https://habr.com/ru/company/pixonic/blog/541622/   
0                   https://habr.com/ru/post/541674/   
0                   https://habr.com/ru/post/541610/   
0     https://habr.com/ru/company/ruvds/blog/541470/   
0  https://habr.com/ru/company/domclick/blog/531254/   
0      https://habr.com/ru/company/otus/blog/541378/   
0                   https://habr.com/ru/post/541356/   
0     https://habr.com/ru/company/ruvds/blog/541228/   
0  https://habr.com/ru/company/selectel/blog/540494/   
0    https://habr.com/ru/company/itelma/blog/540784/   
0  https://habr.com/ru/company/globalsign/blog/54...   
0  https://habr.com/ru/company/selectel/blog/541284/   
0     https://habr.com/ru/company/ruvds/blog/541270/   

                                                text  
0  \n\n\r\n–ú–Ω–µ –≤—Å–µ–≥–¥–∞ —Ö–æ—Ç–µ–ª–æ—Å—å –∑–∞–ø–æ–ª—É—á–∏—Ç—å —ç–∫—Ä–∞–Ω –Ω...  
0  \n–í—ã –∑–∞–º–µ—á–∞–ª–∏, —á—Ç–æ –∑–∞ 20 –ª–µ—Ç –¥–æ–ª—è Linux —Å–∏—Å—Ç–µ–º...  
0  \n–ß–∏–ø Apple M1, –¥–æ—Å—Ç—É–ø–Ω—ã–π –≤ –Ω–æ–≤—ã—Ö MacBook Air,...  
0  \n\n\r\n–ú–Ω–µ –≤—Å–µ–≥–¥–∞ —Ö–æ—Ç–µ–ª–æ—Å—å –∑–∞–ø–æ–ª—É—á–∏—Ç—å —ç–∫—Ä–∞–Ω –Ω...  
0  \n–í—ã –∑–∞–º–µ—á–∞–ª–∏, —á—Ç–æ –∑–∞ 20 –ª–µ—Ç –¥–æ–ª—è Linux —Å–∏—Å—Ç–µ–º...  
0  \n–ß–∏–ø Apple M1, –¥–æ—Å—Ç—É–ø–Ω—ã–π –≤ –Ω–æ–≤—ã—Ö MacBook Air,...  
0  \n      –Ø –Ω–µ –≤–µ—Ä—é –≤ —Ç–µ–æ—Ä–∏—é –∑–∞–≥–æ–≤–æ—Ä–∞ –ø—Ä–æ–∏–∑–≤–æ–¥–∏—Ç...  
0  \n–û–¥–Ω–æ–π –∏–∑ –∑–∞–¥–∞—á —Å–∏—Å—Ç–µ–º –ò–¢-–º–æ–Ω–∏—Ç–æ—Ä–∏–Ω–≥–∞ —è–≤–ª—è–µ—Ç—Å...  
0  \n\r\n20 –ª–µ—Ç ‚Äî –±–æ–ª—å—à–æ–π —Å—Ä–æ–∫, –¥–∞–∂–µ Windows –Ω–µ —Å...  
0  \n      –†–∞–Ω–æ –∏–ª–∏ –ø–æ–∑–¥–Ω–æ –º–∞–ª–µ–Ω—å–∫–∏–µ –ø—Ä–∏–ª–æ–∂–µ–Ω–∏—è —Ä...  
0  \n–í—Å–µ–º –ø—Ä–∏–≤–µ—Ç! –í —ç—Ç–æ–π —Å—Ç–∞—Ç—å–µ —è –ø–æ—Å—Ç–∞—Ä–∞—é—Å—å –æ–ø–∏—Å...  
0  \n–í—Å–µ –º—ã —Å–ª—ã—à–∞–ª–∏ –±–∞–π–∫—É: –µ—Å–ª–∏ –≤ Windows 

### –î–æ–ø–æ–ª–Ω–∏—Ç–µ–ª—å–Ω–∞—è —á–∞—Å—Ç—å (–Ω–µ–æ–±—è–∑–∞—Ç–µ–ª—å–Ω–∞—è)

–£–ª—É—á—à–∏—Ç—å —Å–∫—Ä–∏–ø—Ç —Ç–∞–∫, —á—Ç–æ–±—ã –æ–Ω –∞–Ω–∞–ª–∏–∑–∏—Ä–æ–≤–∞–ª –Ω–µ —Ç–æ–ª—å–∫–æ preview-–∏–Ω—Ñ–æ—Ä–º–∞—Ü–∏—é —Å—Ç–∞—Ç—å–∏, –Ω–æ –∏ –≤–µ—Å—å —Ç–µ–∫—Å—Ç —Å—Ç–∞—Ç—å–∏ —Ü–µ–ª–∏–∫–æ–º.

–î–ª—è —ç—Ç–æ–≥–æ –ø–æ—Ç—Ä–µ–±—É–µ—Ç—Å—è –ø–æ–ª—É—á–∞—Ç—å —Å—Ç—Ä–∞–Ω–∏—Ü—ã —Å—Ç–∞—Ç–µ–π –∏ –∏—Å–∫–∞—Ç—å –ø–æ —Ç–µ–∫—Å—Ç—É –≤–Ω—É—Ç—Ä–∏ —ç—Ç–æ–π —Å—Ç—Ä–∞–Ω–∏—Ü—ã.  

–ò—Ç–æ–≥–æ–≤—ã–π –¥–∞—Ç–∞—Ñ—Ä–µ–π–º —Ñ–æ—Ä–º–∏—Ä–æ–≤–∞—Ç—å —Å–æ —Å—Ç–æ–ª–±—Ü–∞–º–∏: `<–¥–∞—Ç–∞> - <–∑–∞–≥–æ–ª–æ–≤–æ–∫> - <—Å—Å—ã–ª–∫–∞> - <—Ç–µ–∫—Å—Ç_—Å—Ç–∞—Ç—å–∏>`


In [91]:
def load_post(post_link):
    req = requests.get(post_link)
    soup = BeautifulSoup(req.text, 'html.parser')
    text = soup.find('div', class_='post__text').text
    return text

In [92]:
def parse_post_(post):
    post_date = parse_date(post.find('span', class_='post__time').text)
    post_link = post.find('a', class_='post__title_link').get('href')
    post_title = post.find('h2', class_='post__title').text
    post_preview = load_post(post_link)
    return {'date': post_date, 'title': post_title, 'link': post_link, 'text': post_preview}

In [93]:
def filter_posts_(posts):
    result = pd.DataFrame()
    for post in posts:
        post = parse_post_(post)
        if set(post['text'].lower().split(' ')).intersection(KEYWORDS):
            result = pd.concat([result, pd.DataFrame([post])])
        time.sleep(0.3)
    return result

In [101]:
# result2 = filter_posts_(posts)
result2

date                                              title  \
0 2021-02-10 12:42:00                    \n–ü–æ–ø—É–ª—è—Ä–Ω—ã–µ –∑–∞–±–ª—É–∂–¥–µ–Ω–∏—è –æ C#\n   
0 2021-02-10 12:36:00  \n–ü—Ä–µ–≤—Ä–∞—â–∞–µ–º —Å—Ç–∞—Ä—ã–π Amazon Kindle –≤ –ø–ª–∞—Ç—Ñ–æ—Ä–º—É ...   
0 2021-02-10 12:12:00                         \n–†–∞—Å–ø–æ–∑–Ω–∞–≤–∞–Ω–∏–µ —Å–∏–º–≤–æ–ª–æ–≤\n   
0 2021-02-10 11:30:00              \n–°—Ä–∞–∂–µ–Ω–∏–µ Linux –Ω–∞ –ø–æ–ø—Ä–∏—â–µ Windows\n   
0 2021-02-10 11:00:00  \nApple M1 —Ö–æ—Ä–æ—à, –Ω–æ –Ω–∞—Å–∫–æ–ª—å–∫–æ –æ–Ω —Ö–æ—Ä–æ—à –≤ —Ç—Ä–∞—Å...   
0 2021-02-10 09:51:00  \nOSCP: –∫–∞–∫ —è —Å–¥–∞–≤–∞–ª —Å–∞–º—ã–π –∏–∑–≤–µ—Å—Ç–Ω—ã–π —ç–∫–∑–∞–º–µ–Ω –ø...   
0 2021-02-10 09:24:00  \n–£–ø—Ä–∞–≤–ª–µ–Ω–∏–µ LCD –∏ OLED –¥–∏—Å–ø–ª–µ—è–º–∏ –Ω–∞ AVR-–∞—Å—Å–µ–º...   
0 2021-02-10 12:42:00                    \n–ü–æ–ø—É–ª—è—Ä–Ω—ã–µ –∑–∞–±–ª—É–∂–¥–µ–Ω–∏—è –æ C#\n   
0 2021-02-10 12:36:00  \n–ü—Ä–µ–≤—Ä–∞—â–∞–µ–º —Å—Ç–∞—Ä—ã–π Amazon Kindle –≤ –ø–ª–∞—Ç—Ñ–æ—Ä–º—É ...   
0 2021-02-10 12:12:00                         \n–†–∞—Å–ø–æ–∑–Ω–∞–≤–∞–Ω–∏–µ —Å–∏–º–≤–æ–ª–æ–≤\n   
0 2021-02-10 11:30:00              \n–°—Ä–∞–∂–µ–Ω–∏–µ Linux –Ω–∞ –ø–æ–ø—Ä–∏—â–µ Windows\n   
0 2021-02-10 11:00:00  \nApple M1 —Ö–æ—Ä–æ—à, –Ω–æ –Ω–∞—Å–∫–æ–ª—å–∫–æ –æ–Ω —Ö–æ—Ä–æ—à –≤ —Ç—Ä–∞—Å...   
0 2021-02-10 09:51:00  \nOSCP: –∫–∞–∫ —è —Å–¥–∞–≤–∞–ª —Å–∞–º—ã–π –∏–∑–≤–µ—Å—Ç–Ω—ã–π —ç–∫–∑–∞–º–µ–Ω –ø...   
0 2021-02-10 09:24:00  \n–£–ø—Ä–∞–≤–ª–µ–Ω–∏–µ LCD –∏ OLED –¥–∏—Å–ø–ª–µ—è–º–∏ –Ω–∞ AVR-–∞—Å—Å–µ–º...   
0 2021-02-09 22:55:00  \n–í—ã—à–µ–ª —Ä–µ–ª–∏–∑ GitLab 13.8 —Å —Ä–µ–¥–∞–∫—Ç–æ—Ä–æ–º –∫–æ–Ω–≤–µ–π–µ...   
0 2021-02-09 19:21:00              \n–°–±—Ä–æ—Å –ø—Ä–æ–±–µ–≥–∞ –¥–ª—è RICOH SP 150SUw\n   
0 2021-02-09 18:29:00  \nGo, –ª–æ–∫–¥–∞—É–Ω –∏ –Ω–æ–≤—ã–π MPD-–∫–ª–∏–µ–Ω—Ç ‚Äî –≤—Å—Ç—Ä–µ—á–∞–π—Ç–µ ...   
0 2021-02-09 17:37:00    \n–†–∞–∑—Ä–∞–±–∞—Ç—ã–≤–∞–µ–º web-site –¥–ª—è –º–∏–∫—Ä–æ–∫–æ–Ω—Ç—Ä–æ–ª–ª–µ—Ä–∞\n   
0 2021-02-09 16:41:00  \n–ò—Å—Ç–æ—Ä–∏—è –∏–Ω—Ç–µ—Ä–ø—Ä–µ—Ç–∏—Ä—É–µ–º–æ—Å—Ç–∏ –≤ —Ä–∞—Å–ø–æ–∑–Ω–∞–≤–∞–Ω–∏–∏ –∏...   
0 2021-02-09 15:24:00              \n–ò—Å—Å–ª–µ–¥–æ–≤–∞–Ω–∏–µ –≤—Ä–µ–¥–æ–Ω–æ—Å–Ω–æ–≥–æ —Ç—Ä–∞—Ñ–∏–∫–∞\n   
0 2021-02-09 14:53:00  \nRoot cause –∞–Ω–∞–ª–∏–∑ –∏–Ω—Ü–∏–¥–µ–Ω—Ç–æ–≤ –Ω–∞ –∫–æ—Ä—Ä–µ–ª—è—Ü–∏—è—Ö ...   
0 2021-02-09 14:07:00     \n–°–∞–º—ã–µ –æ–∂–∏–¥–∞–µ–º—ã–µ –∏–≥—Ä–æ–≤—ã–µ –Ω–æ—É—Ç–±—É–∫–∏ 2021 –≥–æ–¥–∞\n   
0 2021-02-09 13:20:00   \n–†–∞–∑—Ä–∞–±–æ—Ç–∫–∞ —É–Ω–∏–≤–µ—Ä—Å–∞–ª—å–Ω–æ–≥–æ —Å—á–µ—Ç—á–∏–∫–∞ –∏–º–ø—É–ª—å—Å–æ–≤\n   
0 2021-02-09 12:12:00  \n20 –ª–µ—Ç Vide–æLAN Client Player: 8 –Ω–µ–æ—á–µ–≤–∏–¥–Ω—ã—Ö...   
0 2021-02-09 12:11:00             \n–ù–∞—Å—Ç—Ä–æ–π–∫–∞ WPA2 Enterprise c RADIUS\n   
0 2021-02-09 12:08:00             \n–ü–∏—à–µ–º –ò–ò –¥–ª—è –∏–≥—Ä—ã –ì–æ–º–æ–∫—É (5 –≤ —Ä—è–¥)\n   
0 2021-02-09 11:42:00  \n10 —Å–µ—Ä–≤–∏—Å–æ–≤ –¥–ª—è –æ—Ä–≥–∞–Ω–∏–∑–∞—Ü–∏–∏ —Å–ø–∏—Å–∫–∞ –¥–µ–ª –∏ –ø–æ–≤...   
0 2021-02-09 11:16:00  \n–°–∞–≥–∞ –æ —Ç–∏–ø–∏–∑–∞—Ü–∏–∏ –∏ —Ç–∞–π–ø—á–µ–∫–∏–Ω–≥–µ –¥–ª—è JavaScript\n   
0 2021-02-09 11:14:00  \n–ú–æ–±–∏–ª—å–Ω–æ–µ —Ç–µ—Å—Ç–∏—Ä–æ–≤–∞–Ω–∏–µ, –∞–≤—Ç–æ–º–∞—Ç–∏–∑–∞—Ü–∏—è —Ç–µ—Å—Ç–∏—Ä...   
0 2021-02-09 11:07:00  \n–ö–∞–∫ –≤ GitHub —Å–º–æ—Ç—Ä–µ—Ç—å –∫—Ä–∞—Å–∏–≤—ã–µ –æ—Ç—á–µ—Ç—ã –æ–± –æ—à–∏...   
0 2021-02-09 10:59:00  \n11 –¥—Ä—É–∑–µ–π Sanic‚Äô–∞ ‚Äì —Å–æ–±–∏—Ä–∞–µ–º –∞—Å–∏–Ω—Ö—Ä–æ–Ω–Ω–æ–µ –≤–µ–±...   
0 2021-02-09 10:09:00  \nMyCompany ‚Äî –±–µ—Å–ø–ª–∞—Ç–Ω–æ–µ –∏ –æ—Ç–∫—Ä—ã—Ç–æ–µ —Ä–µ—à–µ–Ω–∏–µ –¥–ª...   
0 2021-02-09 10:01:00  \nVR/AR –∏ –¥—Ä—É–≥–∏–µ —Ç–µ—Ö–Ω–æ–ª–æ–≥–∏–∏ –≤ —Ñ–∏–ª—å–º–µ ¬´–ü–µ—Ä–≤–æ–º—É ...   
0 2021-02-09 02:28:00  \n–ö–∞–∫ —Å–æ–∑–¥–∞–≤–∞–ª—Å—è –æ—Ç–µ—á–µ—Å—Ç–≤–µ–Ω–Ω—ã–π –±—Ä–∞—É–∑–µ—Ä —Å –±–æ–∫–æ–≤...   
0 2021-02-09 01:32:00                       \nCalico Enterprise: –æ–±–∑–æ—Ä\n   
0 2021-02-09 01:25:00  \n–ü–æ–ª—É—á–∞–µ–º –±–µ—Å–ø–ª–∞—Ç–Ω—ã–µ —Å–µ—Ä–≤–µ—Ä–∞ –≤ Oracle Cloud F...   
0 2021-02-09 00:53:00                        \n–ß—Ç–æ —Ç–∞–∫–æ–µ —Ñ–∏–ª—å—Ç—Ä –ë–ª—É–º–∞?\n   
0 2021-02-08 22:08:00  \n–°–æ–∑–¥–∞–Ω–∏–µ —Ç—Ä–µ–∫–µ—Ä–∞ –ø—É–ª—å—Å–æ–∫—Å–∏–º–µ—Ç—Ä–∏–∏ —Å –∏—Å–ø–æ–ª—å–∑–æ–≤...   
0 2021-02-08 20:36:0

## –ó–∞–¥–∞–Ω–∏–µ 2.

### –û–±—è–∑–∞—Ç–µ–ª—å–Ω–∞—è —á–∞—Å—Ç—å

–ù–∞–ø–∏—Å–∞—Ç—å —Å–∫—Ä–∏–ø—Ç, –∫–æ—Ç–æ—Ä—ã–π –±—É–¥–µ—Ç –ø—Ä–æ–≤–µ—Ä—è—Ç—å —Å–ø–∏—Å–æ–∫ e-mail –∞–¥—Ä–µ—Å–æ–≤ –Ω–∞ —É—Ç–µ—á–∫—É –ø—Ä–∏ –ø–æ–º–æ—â–∏ —Å–µ—Ä–≤–∏—Å–∞ [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
–°–ø–∏—Å–æ–∫ email-–æ–≤ –∑–∞–¥–∞–µ–º –ø–µ—Ä–µ–º–µ–Ω–Ω–æ–π –≤ –Ω–∞—á–∞–ª–µ –∫–æ–¥–∞:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

–í –∏—Ç–æ–≥–µ –¥–æ–ª–∂–µ–Ω —Ñ–æ—Ä–º–∏—Ä–æ–≤–∞—Ç—å—Å—è –¥–∞—Ç–∞—Ñ—Ä–µ–π–º —Å–æ —Å—Ç–æ–ª–±—Ü–∞–º–∏: `<–ø–æ—á—Ç–∞> - <–¥–∞—Ç–∞ —É—Ç–µ—á–∫–∏> - <–∏—Å—Ç–æ—á–Ω–∏–∫ —É—Ç–µ—á–∫–∏> - <–æ–ø–∏—Å–∞–Ω–∏–µ —É—Ç–µ—á–∫–∏>`  

**–ü–æ–¥—Å–∫–∞–∑–∫–∞**: —Å–µ—Ä–≤–∏—Å —Ä–∞–±–æ—Ç–∞–µ—Ç –ø—Ä–∏ –ø–æ–º–æ—â–∏ "—Å–∫—Ä—ã—Ç–æ–≥–æ" API. –í–Ω–∏–º–∞—Ç–µ–ª—å–Ω–æ –∏–∑—É—á–∏—Ç–µ post-–∑–∞–ø—Ä–æ—Å—ã.

In [98]:
EMAIL = ['xxx@ya.ru', 'yyy@gmail.com']

In [116]:
api_url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
    'Content-Type': 'application/json;charset=UTF-8',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}

In [117]:
response = requests.post(api_url, json={'emailAddresses': EMAIL}, headers=headers)

In [126]:
data = response.json()
data

{'breaches': {'3883': {'breachId': 3883,
   'site': 'brainybatch.com',
   'recordsCount': 872730,
   'description': "In January 2017, BrainyBatch Technologies' user database was allegedly breached. The stolen data contains over 873,000 records including email addresses, passwords, and full names. The compromised data is being shared privately on the darknet.",
   'publishDate': '2017-04-20T00:00:00Z',
   'statistics': {'usernames': 31188, 'passwords': 872730, 'emails': 872514}},
  '16962': {'breachId': 16962,
   'site': 'cprewritten.net',
   'recordsCount': 1687049,
   'description': "In July 2019, virtual world Club Penguin Rewritten's database was allegedly breached. The stolen data contains usernames, passwords, email addresses and additional personal information. This breach is being publicly shared on the internet.",
   'publishDate': '2019-09-26T00:00:00Z',
   'statistics': {'usernames': 1687014,
    'passwords': 1558530,
    'emails': 1685691}},
  '16591': {'breachId': 16591,
  

In [133]:
result = pd.DataFrame()

for email, breach_ids in data['summary'].items():
    for breach_id in breach_ids['breaches']:
        breach_id = str(breach_id)
        row = {
            'email': email,
            'date': data['breaches'][breach_id]['publishDate'],
            'site': data['breaches'][breach_id]['site'],
            'desc': data['breaches'][breach_id]['description']
        }
        result = pd.concat([result, pd.DataFrame([row])])

result

email                  date                site  \
0       xxx@ya.ru  2016-11-07T00:00:00Z      ir.netease.com   
0       xxx@ya.ru  2020-07-30T00:00:00Z            hurb.com   
0       xxx@ya.ru  2019-03-28T00:00:00Z    verifications.io   
0       xxx@ya.ru  2020-05-21T00:00:00Z              vk.com   
0       xxx@ya.ru  2016-11-01T00:00:00Z              qip.ru   
..            ...                   ...                 ...   
0   yyy@gmail.com  2017-01-16T00:00:00Z  catchitenglish.com   
0   yyy@gmail.com  2017-10-29T00:00:00Z       jobstreet.com   
0   yyy@gmail.com  2017-05-26T00:00:00Z             leet.cc   
0   yyy@gmail.com  2016-12-05T00:00:00Z   classroomcash.org   
0   yyy@gmail.com  2017-03-24T00:00:00Z           youku.com   

                                                 desc  
0   In October 2015, the Chinese internet and gami...  
0   In March 2019, the Brazilian travel agency Her...  
0   Big data e-mail verification platform verifica...  
0   At some time in 2020, the Russian social netwo...  
0   In 2011, Russian instant messaging service pro...  
..                                                ...  
0   In September 2016, CatchItEnglish.com was brea...  
0   Online recruitment site JobStreet was allegedl...  
0   In February 2016, LEET's user database was all...  
0   At an unknown date ClassroomCash.org was hacke...  
0   Youku is a large Chinese video content company...  

[137 rows x 4 columns]

### –î–æ–ø–æ–ª–Ω–∏—Ç–µ–ª—å–Ω–∞—è —á–∞—Å—Ç—å (–Ω–µ–æ–±—è–∑–∞—Ç–µ–ª—å–Ω–∞—è)

–ù–∞–ø–∏—Å–∞—Ç—å —Å–∫—Ä–∏–ø—Ç, –∫–æ—Ç–æ—Ä—ã–π –±—É–¥–µ—Ç –ø–æ–ª—É—á–∞—Ç—å 50 –ø–æ—Å–ª–µ–¥–Ω–∏—Ö –ø–æ—Å—Ç–æ–≤ —É–∫–∞–∑–∞–Ω–Ω–æ–π –≥—Ä—É–ø–ø—ã –≤–æ –í–∫–æ–Ω—Ç–∞–∫—Ç–µ.  
–î–æ–∫—É–º–µ–Ω—Ç–∞—Ü–∏—è –∫ API VK: https://vk.com/dev/methods
, –≤–∞–º –ø–æ–º–æ–∂–µ—Ç –º–µ—Ç–æ–¥ [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = –£–î–ê–õ–Ø–ô–¢–ï –í –í–ï–†–°–ò–ò –î–õ–Ø –ü–†–û–í–ï–†–ö–ò, –ù–ê GITHUB –ù–ï –í–´–ö–õ–ê–î–´–í–ê–¢–¨  
```

–í –∏—Ç–æ–≥–µ –¥–æ–ª–∂–µ–Ω —Ñ–æ—Ä–º–∏—Ä–æ–≤–∞—Ç—å—Å—è –¥–∞—Ç–∞—Ñ—Ä–µ–π–º —Å–æ —Å—Ç–æ–ª–±—Ü–∞–º–∏: `<–¥–∞—Ç–∞ –ø–æ—Å—Ç–∞> - <—Ç–µ–∫—Å—Ç –ø–æ—Å—Ç–∞>`

In [142]:
api_url = 'https://api.vk.com/method/wall.get'
params = {
    'domain': 'netology',
    'access_token': '',
    'v': '5.126',
    'count': 50
}

In [143]:
response = requests.get(api_url, params=params)

In [146]:
data = pd.read_json(json.dumps(response.json()['response']['items']))

In [148]:
data[['date', 'text']]

date                                               text
0  2021-02-05 07:53:00  –ì–¥–µ —Ä–∞–±–æ—Ç–∞—Ç—å –¥–∏–∑–∞–π–Ω–µ—Ä—É? ü§î \n–ì—Ä–∞—Ñ–∏—á–µ—Å–∫–∏–π –¥–∏–∑–∞–π–Ω...
1  2021-02-10 07:53:00  üìÜ 15 —Ñ–µ–≤—Ä–∞–ª—è —Å—Ç–∞—Ä—Ç—É–µ—Ç –±–µ—Å–ø–ª–∞—Ç–Ω—ã–π –∫—É—Ä—Å ¬´–†–∞–∑—Ä–∞–±–æ...
2  2021-02-09 14:57:00  –†–∞–∑, –¥–≤–∞, —Ç—Ä–∏ ‚Äî —ç–º–æ—Ü–∏—è –∑–∞–º—Ä–∏ ü§™ \n \n–≠–º–æ—Ü–∏–æ–Ω–∞–ª—å...
3  2021-02-09 07:27:00  –ö—Ç–æ —Å—Ç–æ–∏—Ç –∑–∞ –≤—Å–µ–º–∏ –ø—Ä–∏–ª–æ–∂–µ–Ω–∏—è–º–∏, –∏–≥—Ä–∞–º–∏ –∏ —Å–∞–π—Ç...
4  2021-02-08 15:09:00  –°–æ–º–Ω–µ–Ω–∏–π –±–æ–ª—å—à–µ –Ω–µ—Ç: –∂–∏–∑–Ω—å –Ω–∞ —É–¥–∞–ª—ë–Ω–∫–µ ‚Äî –µ—Å—Ç—å ...
5  2021-02-08 07:38:00  –ï—Å–ª–∏ –∏–Ω—Ç–µ—Ä–Ω–µ—Ç —Å—Ä–∞–≤–Ω–∏—Ç—å —Å —ç–∫–∑–æ—Ç–∏—á–µ—Å–∫–æ–π —Å—Ç—Ä–∞–Ω–æ–π,...
6  2021-02-07 13:07:00  *–ø–∞—Ä—Ç–Ω—ë—Ä—Å–∫–∏–π –ø–æ—Å—Ç*\n\n25-26 –º–∞—Ä—Ç–∞ —Å–æ—Å—Ç–æ–∏—Ç—Å—è –∫–æ...
7  2021-02-07 11:37:00  –ú–æ–∂–Ω–æ –ø–æ–ª–æ–∂–∏—Ç—å –∫–æ—à–µ–ª—ë–∫ –Ω–∞ –ø–æ–¥–æ–∫–æ–Ω–Ω–∏–∫ –≤ –ø–æ–ª–Ω–æ–ª—É...
8  2021-02-06 12:07:00  *–ø–∞—Ä—Ç–Ω—ë—Ä—Å–∫–∏–π –ø–æ—Å—Ç* \n \n–ò–∑–¥–∞—Ç–µ–ª—å—Å—Ç–≤–æ –ú–ò–§ –∑–∞–ø—É—Å...
9  2021-02-06 08:07:00  –°–æ–±—Ä–∞–ª–∏ –ø–æ–¥–±–æ—Ä–∫—É —Å—Ç–∞—Ç–µ–π, –≤ –ø–æ–º–æ—â—å –Ω–∞—á–∏–Ω–∞—é—â–∏–º S...
10 2021-02-05 14:27:00  –í —ç—Ñ–∏—Ä–µ #–ø—è—Ç–Ω–∏—á–Ω–∞—è–±–µ—Å–µ–¥–∫–∞. –ó–¥–µ—Å—å –º—ã –æ–±—Å—É–∂–¥–∞–µ–º,...
11 2021-02-04 14:53:00  –†–∞–±–æ—á–∏–µ —Å–æ–≤–µ—â–∞–Ω–∏—è –º–æ–≥—É—Ç –±—ã—Ç—å –Ω–∞—Å—Ç–æ—è—â–∏–º–∏ –ø–æ–∂–∏—Ä–∞...
12 2021-02-04 08:13:00  –ß—Ç–æ–±—ã –Ω–µ –ø–æ–≤–µ—Å–∏—Ç—å –Ω–∞ –¥–≤–µ—Ä–∏ —Ç–∞–±–ª–∏—á–∫—É ¬´–∑–∞–∫—Ä—ã—Ç–æ¬ª,...
13 2021-02-03 14:38:00  –ò–Ω–æ–≥–¥–∞ –±—ã–≤–∞–µ—Ç —Å–ª–æ–∂–Ω–æ —Ä–µ—à–∏—Ç—å—Å—è –Ω–∞ –ø–µ—Ä–µ–º–µ–Ω—ã –≤ –∂–∏...
14 2021-02-03 07:37:00  üî• 8 —Ñ–µ–≤—Ä–∞–ª—è —Å—Ç–∞—Ä—Ç—É–µ—Ç –±–µ—Å–ø–ª–∞—Ç–Ω—ã–π –∫—É—Ä—Å-–ø—É—Ç–µ–≤–æ–¥–∏—Ç...
15 2021-02-02 15:07:00  –ö–æ–Ω—Å–ø–µ–∫—Ç—ã —ç–∫–æ–Ω–æ–º—è—Ç –º–∞—Å—Å—É –≤—Ä–µ–º–µ–Ω–∏ –≤ –±—É–¥—É—â–µ–º: –Ω–µ...
16 2021-02-02 07:53:00  –ë–æ–ª–µ–µ –ø–æ–ª–æ–≤–∏–Ω—ã —Ä–∞–±–æ—Ç–æ–¥–∞—Ç–µ–ª–µ–π —Ü–µ–Ω—è—Ç —É–º–µ–Ω–∏–µ –æ–±—â–∞...
17 2021-02-01 15:22:00  –ù–æ–≤—ã–µ —Ç—Ä–µ–Ω–¥—ã –≤ –¥–∏–∑–∞–π–Ω–µ –∏–Ω—Ç–µ—Ä—Ñ–µ–π—Å–æ–≤, –Ω–µ–æ–±—ã—á–Ω—ã–µ ...
18 2021-02-01 07:44:00  –ú—ã –≤–µ—Ä–∏–º, —á—Ç–æ –≤ Data Science –º–æ–∂–µ—Ç —Ä–∞–±–æ—Ç–∞—Ç—å –∫–∞...
19 2021-01-31 11:27:00  –£–º–µ–Ω–∏–µ –±—ã—Ç—å –ø—Ä–∏—è—Ç–Ω—ã–º —Å–æ–±–µ—Å–µ–¥–Ω–∏–∫–æ–º, –∫–æ—Ç–æ—Ä–æ–º—É –¥–æ...
20 2021-01-30 12:41:00  ¬´–í–∞—à –∑–≤–æ–Ω–æ–∫ –æ—á–µ–Ω—å –≤–∞–∂–µ–Ω –¥–ª—è –Ω–∞—Å¬ª üìû \n–î–æ–ª–≥–æ–µ –æ–∂...
21 2021-01-29 14:44:00  –í —ç—Ñ–∏—Ä–µ #–ø—è—Ç–Ω–∏—á–Ω–∞—è–±–µ—Å–µ–¥–∫–∞. –ó–¥–µ—Å—å –º—ã –æ–±—Å—É–∂–¥–∞–µ–º,...
22 2021-01-29 07:44:00  –¢–æ—á–Ω–æ —Ä–µ—à–∏–ª–∏, –∫–µ–º —Ö–æ—Ç–∏—Ç–µ —Å—Ç–∞—Ç—å –≤ –¥–∏–¥–∂–∏—Ç–∞–ª–µ, –Ω–æ...
23 2021-01-28 14:51:00  –ö–∞–∫ —Å–æ—Å—Ç–∞–≤–∏—Ç—å –∫–æ–Ω—Ç–µ–Ω—Ç-–ø–ª–∞–Ω –¥–ª—è —Å–æ—Ü–∏–∞–ª—å–Ω—ã—Ö —Å–µ—Ç–µ...
24 2021-01-28 07:57:00  –ö–∞–∫ –Ω–∞—É—á–∏—Ç—å —Ä–æ–±–æ—Ç–∞ –≥–æ—Ç–æ–≤–∏—Ç—å –æ–ª–∏–≤—å–µ, –∞ –Ω–µ–π—Ä–æ—Å–µ—Ç...
25 2021-01-27 14:13:00  –°–µ–≥–æ–¥–Ω—è –∑–∞–¥–∞—á–∫–∞ –Ω–∞ –≤–Ω–∏–º–∞—Ç–µ–ª—å–Ω–æ—Å—Ç—å: –¥–∞–≤–∞–π—Ç–µ —É–∑–Ω...
26 2021-01-27 07:13:00  üöÄ –ó–∞–ø—É—Å—Ç–∏–ª–∏ –±–µ—Å–ø–ª–∞—Ç–Ω—ã–π –∫—É—Ä—Å ¬´–û—Å–Ω–æ–≤—ã Figma¬ª \n ...
27 2021-01-26 14:48:00  –ö–æ–≥–¥–∞-—Ç–æ –∏–Ω—Ç–µ—Ä–Ω–µ—Ç —Å–æ—Å—Ç–æ—è–ª –∏–∑ —Ç–µ–∫—Å—Ç–æ–≤ –∏ –ø—Ä–æ—Å—Ç—ã—Ö...
28 2021-01-26 07:53:00  –ö–∏–±–µ—Ä–ø–∞–Ω–∫ ‚Äî —Ñ–∞–Ω—Ç–∞—Å—Ç–∏–∫–∞ –∏–ª–∏ –Ω–∞—à–µ –≤–æ–∑–º–æ–∂–Ω–æ–µ –±—É–¥—É...
29 2021-01-25 14:53:00  –ê–Ω–∞–ª–∏–∑ –¶–ê, –∑–Ω–∞–Ω–∏–µ –æ—Å–Ω–æ–≤ –ø—Å–∏—Ö–æ–ª–æ–≥–∏–∏ –∏ –±–∞–∑–æ–≤—ã—Ö –ø...
30 2021-01-25 07:27:00  –ö–∞—Ä—å–µ—Ä–∞ —Å–µ–≥–æ–¥–Ω—è ‚Äî –Ω–µ –ø—Ä—è–º–æ–π –∏ –ø–æ–Ω—è—Ç–Ω—ã–π –ø—É—Ç—å –∏–∑...
31 2021-01-24 11:15:00  –ö–∞–∫ –≤–ª–∏—è—Ç—å –Ω–∞ –ª—é–¥–µ–π —Å –ø–æ–º–æ—â—å—é —Å–ª–æ–≤ –∏ –ø–æ—á–µ–º—É —É–º...
32 2021-01-23 11:22:00  –£–µ—Ö–∞—Ç—å –∏ (–Ω–µ) –≤–µ—Ä–Ω—É—Ç—å—Å—è: —Å–æ–±—Ä–∞–ª–∏ —Å—Ç–∞—Ç—å–∏ –¥–ª—è —Ç–µ...
33 2021-01-22 15:10:00  –í —ç—Ñ–∏—Ä–µ #–ø—è—Ç–Ω–∏—á–Ω–∞—è–±–µ—Å–µ–¥–∫–∞. –ó–¥–µ—Å—å –º—ã –æ–±—Å—É–∂–¥–∞–µ–º,...
34 2021-01-22 07:22:00  –£—á–∏–º—Å—è –Ω–æ–≤–æ–º—É –≤ —É–¥–æ–±–Ω–æ–µ –≤—Ä–µ–º—è: —Å–æ–±—Ä–∞–ª–∏ –±–µ—Å–ø–ª–∞—Ç...
35 2021-01-21 14:37:00 

#### –ü–†–ò–ú–ï–ß–ê–ù–ò–ï
–î–æ–º–∞—à–Ω–µ–µ –∑–∞–¥–∞–Ω–∏–µ —Å–¥–∞–µ—Ç—Å—è —Å—Å—ã–ª–∫–æ–π –Ω–∞ —Ä–µ–ø–æ–∑–∏—Ç–æ—Ä–∏–π [GitHub](https://github.com/).
–ù–µ —Å–º–æ–∂–µ–º –ø—Ä–æ–≤–µ—Ä–∏—Ç—å –∏–ª–∏ –ø–æ–º–æ—á—å, –µ—Å–ª–∏ –≤—ã –ø—Ä–∏—à–ª–µ—Ç–µ:
- —Ñ–∞–π–ª—ã;
- –∞—Ä—Ö–∏–≤—ã;
- —Å–∫—Ä–∏–Ω—à–æ—Ç—ã –∫–æ–¥–∞.

–í—Å–µ –æ–±—Å—É–∂–¥–µ–Ω–∏—è –∏ –∫–æ–Ω—Å—É–ª—å—Ç–∞—Ü–∏–∏ –ø–æ –≤—ã–ø–æ–ª–Ω–µ–Ω–∏—é –¥–æ–º–∞—à–Ω–µ–≥–æ –∑–∞–¥–∞–Ω–∏—è –≤–µ–¥—É—Ç—Å—è —Ç–æ–ª—å–∫–æ –Ω–∞ —Å–æ–æ—Ç–≤–µ—Ç—Å—Ç–≤—É—é—â–µ–º –∫–∞–Ω–∞–ª–µ –≤ slack.

##### –ö–∞–∫ –ø—Ä–∞–≤–∏–ª—å–Ω–æ –∑–∞–¥–∞–≤–∞—Ç—å –≤–æ–ø—Ä–æ—Å—ã –∞—Å–ø–∏—Ä–∞–Ω—Ç–∞–º, –ø—Ä–µ–ø–æ–¥–∞–≤–∞—Ç–µ–ª—è–º –∏ –∫–æ–ª–ª–µ–≥–∞–º?
–ü—Ä–µ–∂–¥–µ —á–µ–º –∑–∞–¥–∞—Ç—å –≤–æ–ø—Ä–æ—Å –Ω–µ–æ–±—Ö–æ–¥–∏–º–æ –ø–æ–ø—Ä–æ–±–æ–≤–∞—Ç—å –Ω–∞–π—Ç–∏ –æ—Ç–≤–µ—Ç —Å–∞–º–æ–º—É –≤ –∏–Ω—Ç–µ—Ä–Ω–µ—Ç–µ. –ù–∞–≤—ã–∫ —Å–∞–º–æ—Å—Ç–æ—è—Ç–µ–ª—å–Ω–æ–≥–æ –ø–æ–∏—Å–∫–∞ –∏–Ω—Ñ–æ—Ä–º–∞—Ü–∏–∏ ‚Äì –æ–¥–∏–Ω –∏–∑ –≤–∞–∂–Ω–µ–π—à–∏—Ö, –∏ –∫–∞–∂–¥—ã–π –ø—Ä–∞–∫—Ç–∏–∫—É—é—â–∏–π —Å–ø–µ—Ü–∏–∞–ª–∏—Å—Ç –ª—é–±–æ–≥–æ —É—Ä–æ–≤–Ω—è —ç—Ç–æ –¥–µ–ª–∞–µ—Ç –∫–∞–∂–¥—ã–π –¥–µ–Ω—å.

–õ—é–±–æ–π –≤–æ–ø—Ä–æ—Å –¥–æ–ª–∂–µ–Ω –±—ã—Ç—å —Å—Ñ–æ—Ä–º—É–ª–∏—Ä–æ–≤–∞–Ω –ø–æ –∞–ª–≥–æ—Ä–∏—Ç–º—É:  
1) –ß—Ç–æ —è –¥–µ–ª–∞—é?  
2) –ö–∞–∫–æ–≥–æ —Ä–µ–∑—É–ª—å—Ç–∞—Ç–∞ —è –æ–∂–∏–¥–∞—é?  
3) –ö–∞–∫ —Ñ–∞–∫—Ç–∏—á–µ—Å–∫–∏–π —Ä–µ–∑—É–ª—å—Ç–∞—Ç –æ—Ç–ª–∏—á–∞–µ—Ç—Å—è –æ—Ç –æ–∂–∏–¥–∞–µ–º–æ–≥–æ?  
4) –ß—Ç–æ —è —É–∂–µ –ø–æ–ø—Ä–æ–±–æ–≤–∞–ª —Å–¥–µ–ª–∞—Ç—å, —á—Ç–æ–±—ã –∏—Å–ø—Ä–∞–≤–∏—Ç—å –ø—Ä–æ–±–ª–µ–º—É?  

–ü–æ –≤–æ–∑–º–æ–∂–Ω–æ—Å—Ç–∏, –ø—Ä–∏–∫—Ä–µ–ø–ª—è–π—Ç–µ –∫ –≤–æ–ø—Ä–æ—Å—É —Å–∫—Ä–∏–Ω—à–æ—Ç—ã, –ª–∏–±–æ —Å—Å—ã–ª–∫–∏ –Ω–∞ –∫–æ–¥. –û—Å—Ç–∞–≤–ª—è–π—Ç–µ —Ç–æ–ª—å–∫–æ –ø—Ä–æ–±–ª–µ–º–Ω—ã–π –∏ –≤–æ—Å–ø—Ä–æ–∏–∑–≤–æ–¥–∏–º—ã–π —É—á–∞—Å—Ç–æ–∫ –∫–æ–¥–∞, –≤—Å–µ —Ä–µ—à–µ–Ω–∏–µ –≤—ã–∫–ª–∞–¥—ã–≤–∞—Ç—å –Ω–µ –¥–æ–ø—É—Å–∫–∞–µ—Ç—Å—è.
